In [1]:
from pathlib import Path
from tqdm import tqdm

import torch
import numpy as np
from PIL import Image
from IPython.display import display

from models import Generator

In [2]:
device = 'cuda' # 'cpu'
LATENT_SIZE = 512
LABEL_SIZE  = 2 # #classes
EMBEDDING_SIZE = 1
RESOLUTION  = 128
# CKPT_PATH = '/home/dgx/yct/StyleGAN2_Pytorch/results/00016-1gpu-itri/checkpoints/ckpt-200000.pt' # es2
CKPT_PATH = '/home/dgx/yct/StyleGAN2_Pytorch/results/00006-1gpu-itri-noaug-finetune/checkpoints/ckpt-297500.pt' # es1
BATCH_SIZE = 40
OUT_DIR = Path('~/yct/samples_0930/').expanduser()
if not OUT_DIR.exists():
    OUT_DIR.mkdir()

In [3]:
ckpt = torch.load(CKPT_PATH)['g_ema']

In [4]:
g_ema = Generator(LATENT_SIZE, LABEL_SIZE, RESOLUTION, embedding_size=EMBEDDING_SIZE, is_training=False).to(device)
g_ema.eval()
g_ema.load_state_dict(ckpt)

<All keys matched successfully>

In [6]:
class Args:
    cfg = 'experiments/itri.yml'
config = update_config(config, Args())

### generate dataset

In [5]:
from tqdm import tqdm
SUB_CLASS = ['NG', 'OK']
for sub in SUB_CLASS:
    (OUT_DIR / sub).mkdir()

idx = 0
for i in range(LABEL_SIZE):
    print(SUB_CLASS[i])
    for b in tqdm(range(2500)):
        noise = [torch.randn(BATCH_SIZE, LATENT_SIZE).to(device)]
        fake_label = torch.full((BATCH_SIZE,), i, dtype=torch.long).to(device)

        samples = g_ema(noise, labels_in=fake_label)[0].cpu().detach().numpy()
        samples = np.clip(samples * 127.5 + 127.5, 0, 255).astype(np.uint8)
        samples = np.moveaxis(samples, 1, -1)

        for j, img in enumerate(samples):
            Image.fromarray(img).save(OUT_DIR / SUB_CLASS[i] / f'{SUB_CLASS[i]}-{str(idx).zfill(6)}.png')
            idx += 1
        
    # grid image
#     grid_img = Image.fromarray(gallery(samples))
#     grid_img.save(OUT_DIR / SUB_CLASS[i] / 'grid.png')
#     display(grid_img)

  0%|          | 0/2500 [00:00<?, ?it/s]

NG


  0%|          | 0/2500 [00:00<?, ?it/s]

OK


100%|██████████| 2500/2500 [28:29<00:00,  1.46it/s]
